In [1]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## 데이터 불러오기

In [2]:
rating_file_path = os.getenv('HOME') + '/data/ml-1m/ratings.dat'
ratings_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(rating_file_path,sep='::',names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 평점 3점 이상만 가져오기

In [3]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size:{orginal_data_size}, filtered_data_size:{filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size/orginal_data_size:.2%}')

orginal_data_size:1000209, filtered_data_size:836478
Ratio of Remaining Data is 83.63%


## rating column 이름을 count로 바꾼다.
### 우리는 평점을 시청횟수로 간주할꺼니까!

In [4]:
ratings.rename(columns={'rating':'count'}, inplace = True)

In [5]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
ratings = ratings.drop(columns = 'timestamp')

In [7]:
ratings.head()

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


## 영화 제목을 보기 위해 메타 데이터를 읽어온다.

In [8]:
movie_file_path = os.getenv('HOME') + '/data/ml-1m/movies.dat'
cols = ['movie_id','title','genre']
movies = pd.read_csv(movie_file_path,sep='::',names=cols,engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
movies['title'] = movies['title'].str.lower()
movies = movies.drop(columns='genre')
movies.tail(10)

,movie_id,title
3873,3943,bamboozled (2000)
3874,3944,bootmen (2000)
3875,3945,digimon: the movie (2000)
3876,3946,get carter (2000)
3877,3947,get carter (1971)
3878,3948,meet the parents (2000)
3879,3949,requiem for a dream (2000)
3880,3950,tigerland (2000)
3881,3951,two family house (2000)
3882,3952,"contender, the (2000)"


## 분석해보자
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)
  
__유저가 시청한 영화보다 가지고 있는 영화 목록이 더 많다.__

In [10]:
ratings['movie_id'].nunique()

3628

In [11]:
movies['movie_id'].nunique()

3883

In [12]:
ratings['user_id'].nunique()

6039

__가장 인기 있는 영화 30개 뽑기__

In [13]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

__유저별 몇 개의 영화를 보았는지에 대한 통계__

In [14]:
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

__유저별 영화 시청 횟수 중앙값에 대한 통계__

In [15]:
user_median = ratings.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 내가 선호하는 영화를 5가지 골라서 rating에 추가해주자

### indexing하기
moives 데이터프레임의 movie_id를 ratings의 movie_id에 인덱싱 해준다.  
메타데이터와 rating 데이터에 공통으로 있는 movie_id로 두 데이터프레임을 머지해주고  
title을 다시 인덱싱해준다.

In [16]:
data = pd.merge(ratings, movies, how='left', on='movie_id')

In [17]:
my_favorite = ['big (1988)', 'ben-hur (1959)', 'wizard of oz, the (1939)', 'last temptation of christ, the (1988)','apollo 13 (1995)']
my_movielist = pd.DataFrame({'user_id':['yoon']*5,'title':my_favorite, 'count':[5,4,5,3,4]})

if not data.isin({'user_id':['yoon']})['user_id'].any():
    data = data.append(my_movielist)

data.tail(10)

,user_id,movie_id,count,title
836473,6040,1090.0,3,platoon (1986)
836474,6040,1094.0,5,"crying game, the (1992)"
836475,6040,562.0,5,welcome to the dollhouse (1995)
836476,6040,1096.0,4,sophie's choice (1982)
836477,6040,1097.0,4,e.t. the extra-terrestrial (1982)
0,yoon,NaN,5,big (1988)
1,yoon,NaN,4,ben-hur (1959)
2,yoon,NaN,5,"wizard of oz, the (1939)"
3,yoon,NaN,3,"last temptation of christ, the (1988)"
4,yoon,NaN,4,apollo 13 (1995)


In [18]:
data_count = ratings.groupby('movie_id')['user_id'].count()
data_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [19]:
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
data.nunique()

user_id     6040
movie_id    3628
count          3
title       3628
dtype: int64

In [21]:
data.tail()

,user_id,movie_id,count,title
0,yoon,NaN,5,big (1988)
1,yoon,NaN,4,ben-hur (1959)
2,yoon,NaN,5,"wizard of oz, the (1939)"
3,yoon,NaN,3,"last temptation of christ, the (1988)"
4,yoon,NaN,4,apollo 13 (1995)


In [22]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    print('user_id colunm indexing OK')
    data['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!')

temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail')
data

user_id colunm indexing OK
title column indexing OK!


,user_id,movie_id,count,title
0,0,1193.0,5,0
1,0,661.0,3,1
2,0,914.0,3,2
3,0,3408.0,4,3
4,0,2355.0,5,4
...,...,...,...,...
0,6039,NaN,5,19
1,6039,NaN,4,6
2,6039,NaN,5,9
3,6039,NaN,3,2446


## CSR Matrix 를 직접 만들어 보자

In [23]:
num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), 
                      shape=(num_user, num_title))
csr_data
csr_data_transpose = csr_data.T

## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜보자

In [24]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [25]:
als_model = AlternatingLeastSquares(factors=200, regularization=0.01,
                                   use_gpu=False, iterations=35,
                                   dtype=np.float32)

In [26]:
als_model.fit(csr_data_transpose)

## 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보자.
  
  
내가 좋아하는 영화  
- 'big (1988)'
- 'ben-hur (1959)'
- 'wizard of oz, the (1939)'
- 'last temptation of christ, the (1988)'
- 'apollo 13 (1995)'

In [34]:
yoon, ben_hur = user_to_idx['yoon'], title_to_idx['ben-hur (1959)']
big = title_to_idx['big (1988)']
oz = title_to_idx['wizard of oz, the (1939)']
last = title_to_idx['last temptation of christ, the (1988)']
aplollo = title_to_idx['apollo 13 (1995)']

yoon_vector, ben_hur_vector = als_model.user_factors[yoon], als_model.item_factors[ben_hur]
big_vector = als_model.item_factors[big]
oz_vector =als_model.item_factors[oz]
last_vector =als_model.item_factors[last]
aplollo_vector =als_model.item_factors[aplollo]

In [37]:
def vector_dot(uvector, movie_vectors):
    for mov in movie_vectors:
        print(np.dot(uvector, mov))

In [47]:
#인덱스를 다시 영화 제목으로 바꿔주는 함수
idx_to_title = {v:k for k,v in title_to_idx.items()}
def get_similar_movie(movie_title: str):
    movie_id = title_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [48]:
movie_vectors=[big_vector, ben_hur_vector, oz_vector, last_vector, aplollo_vector]
vector_dot(yoon_vector, movie_vectors)

0.5087271
0.44561523
0.6349724
0.0849483
0.5166879


내가 가장 선호하는 영화는 벡터곱 결과 0.63으로 wizard of oz, the (1939)

In [49]:
#본적 없는 영화 toy story (1995)
toy_story = title_to_idx['toy story (1995)']
toy_story_vector = als_model.item_factors[toy_story]
np.dot(yoon_vector, toy_story_vector)

-0.020081293

## 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [59]:
#내가 좋아하는 영화 목록에 있던 오즈의 마법사와 유사한 영화를 뽑아보면
#초콜릿 공장이 바로 다음 목록으로 나오는 것을 확인할 수 있다.
get_similar_movie('wizard of oz, the (1939)')

['wizard of oz, the (1939)',
 'willy wonka and the chocolate factory (1971)',
 'to kill a mockingbird (1962)',
 "singin' in the rain (1952)",
 'snow white and the seven dwarfs (1937)',
 'deliverance (1972)',
 'fantasia (1940)',
 'west side story (1961)',
 'citizen kane (1941)',
 'passion in the desert (1998)']

#### 내가 가장 선호하는 영화를 기반으로 비슷한 영화를 출력시켜보면
#### 초콜릿공장을 가장 좋아할 것으로 예측/추천 할 수 있을 것이다.

## 내가 가장 좋아할 만한 영화들을 추천받아 보자
내가 선호할 것 같은 영화 10개를 추천받고  
예상 선호도? 상관 정도를 출력시킨다.

In [50]:
similar_movies = als_model.recommend(yoon, csr_data, N=10, filter_already_liked_items=True)
similar_movies

[(678, 0.30975696),
 (26, 0.2973113),
 (527, 0.25062478),
 (980, 0.2501316),
 (582, 0.2422399),
 (365, 0.240998),
 (116, 0.19877598),
 (248, 0.18881266),
 (657, 0.1795411),
 (170, 0.1766638)]

In [51]:
#인덱스 값을 영화제목으로 다시 바꿔준다.
[idx_to_title[i[0]] for i in similar_movies]

['willy wonka and the chocolate factory (1971)',
 'e.t. the extra-terrestrial (1982)',
 'batman (1989)',
 'seven samurai (the magnificent seven) (shichinin no samurai) (1954)',
 'beetlejuice (1988)',
 'splash (1984)',
 'dances with wolves (1990)',
 'good will hunting (1997)',
 'excalibur (1981)',
 'being john malkovich (1999)']

### 가장 선호도가 높았던 세 영화에 대해서 알아보자  
1. willy wonka and the chocolate factory (1971) : 0.392
2. e.t. the extra-terrestrial (1982)            : 0.297
3. batman (1989)                           : 0.250

In [56]:
willy = title_to_idx['willy wonka and the chocolate factory (1971)']
explain = als_model.explain(user, csr_data, itemid=willy)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('big (1988)', 0.17824572315049841),
 ('wizard of oz, the (1939)', 0.1545268397630636),
 ('last temptation of christ, the (1988)', 0.015087642691221599),
 ('ben-hur (1959)', 0.009163706612313752),
 ('apollo 13 (1995)', -0.05108582496109432)]

In [57]:
et = title_to_idx['e.t. the extra-terrestrial (1982)']
explain = als_model.explain(user, csr_data, itemid=et)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('big (1988)', 0.1849337907155264),
 ('wizard of oz, the (1939)', 0.12739093923599282),
 ('apollo 13 (1995)', 0.03516376707018306),
 ('last temptation of christ, the (1988)', -0.015536921543453952),
 ('ben-hur (1959)', -0.0377249217547883)]

In [58]:
beetle = title_to_idx['beetlejuice (1988)']
explain = als_model.explain(user, csr_data, itemid=beetle)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('big (1988)', 0.30564640352963357),
 ('ben-hur (1959)', 0.0003398720827938271),
 ('last temptation of christ, the (1988)', -0.016439789796566587),
 ('apollo 13 (1995)', -0.023812015535447928),
 ('wizard of oz, the (1939)', -0.025512331556483744)]

### 고찰 및 느낀점
좀 더 깊이 있게 어떻게 알고리즘들이 구현되고 우리가 사용한 라이브러리와 메소드들이  
내부적으로 어떤 연산을 거치는지는 더 많은 공부가 필요하겠지만  
데이터를 어떻게 다루고 어떤식으로 코드를 구현해야 기본적인 추천시스템이 동작하는지 경험해볼 수 있는 시간이었다.  
  
한가지 흥미로웠던 것은 실제 예측/추천 되어 나온 결과가 명확한 정답이 아니라 이용자의 판단에 의해 이루어지는 것이기 때문에  
더 많은 정보 즉 추천을 위해 사용자의 정보와 상품의 정보 그리고 또 다른 사용자들의 정보들이 모두 잘 이용되어야  
좋은 시스템을 갖출 수 있다는 것이었다.  
  
코드 적으로는 시간이 많이 부족해 클래스를 구현하거나 함수를 만들어서 좀 더 간략하고 효율적으로  
코드를 만들어 보고 싶었지만 그러지 못해서 아쉬움이 남는다.